Using the opensource data from bytedance!
[1] Qiuqiang Kong, Bochen Li, Jitong Chen, and Yuxuan Wang. "GiantMIDI-Piano: A large-scale MIDI dataset for classical piano music." arXiv preprint arXiv:2010.07061 (2020). https://arxiv.org/pdf/2010.07061

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
import seaborn as sns


Due to the formatting of the txt files, i would have to delete part of the txt to be able to continue.

In [2]:
# import os
# import pandas as pd

# # Directory containing the text files
# directory = 'datasetsound'

# # List all files in the directory
# files = os.listdir(directory)

# # Read each text file into a DataFrame and print the head
# dataframes = {}  # Dictionary to store DataFrames

# for file in files:
#     if file.endswith('.txt'):  # Check if the file is a text file
#         file_path = os.path.join(directory, file)
#         df = pd.read_csv(file_path, delimiter='\t')  # Assuming tab-separated data; Change delimiter as per your file
        
#         # Store each DataFrame in a dictionary with file names as keys
#         dataframes[file] = df


#         # Print the head of each DataFrame
#         print(df.head())
#         print("-" * 50)  # Separator for better readability

#         # Start analysing
#         headers = df.columns.tolist()
#         print("Headers:", headers)
#         first_row = df.iloc[0]
#         print("First Row:")
#         print(df.iloc[0])
#         print(df.iloc[1])

#         #testing purposes
#         break
# #        new_headers = ['alignID', 'alignOntime', 'alignSitch', 'alignPitch', 'alignOnvel', 'refID', 'refOntime', 'refSitch', 'refPitch', 'refOnvel']


In [9]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Directory containing the text files
directory = 'datasetsound'

# List all files in the directory
files = os.listdir(directory)

# Dictionary to store DataFrames
dataframes = {}

# Define specific headers for the columns
custom_headers = ['alignID', 'alignOntime', 'alignSitch', 'alignPitch', 'alignOnvel', 'refID', 'refOntime', 'refSitch', 'refPitch', 'refOnvel']

# Read each text file into a DataFrame
for file in files:
    if file.endswith('.txt'):  # Check if the file is a text file
        file_path = os.path.join(directory, file)
        
        # Read the file, skip the first line, and assign headers
        with open(file_path, 'r') as f:
            # Skip the first line
            lines = f.readlines()[1:]
            data = [line.strip().split('\t') for line in lines]
            
            # Convert the data into a DataFrame with specific headers
            df = pd.DataFrame(data, columns=custom_headers)
        
        # Store the DataFrame in the dictionary
        dataframes[file] = df
        
        # Display the DataFrame shape for understanding
        print(f"Shape for {file}: {df.shape}")
        print(f"DataFrame for {file}:")
        print(df.head(5))
        print("-" * 50)  # Separator for better readability

all_sitch_values = []

for df in dataframes.values():
    all_sitch_values.extend(df['alignSitch'].unique())

unique_sitch_array = np.sort(np.array(list(set(all_sitch_values))))

def train_lstm_on_dataset(data):
    sequence_length = 10  # Define the sequence length

    data = data['alignSitch'].values.astype('float32')  # Convert data to float32

    sequences = []
    next_sitch = []

    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        next_sitch.append(data[i + sequence_length])

    X = np.array(sequences)
    y = np.array(next_sitch)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(50))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, epochs=5, batch_size=32)

    loss = model.evaluate(X_test, y_test)
    print(f"Test Loss: {loss}")

    return model

# Dictionary to store trained models
trained_models = {}

# Train LSTM model on each DataFrame
for file, df in dataframes.items():
    indices = np.searchsorted(unique_sitch_array, df['alignSitch'])
    df['alignSitch'] = indices
    model = train_lstm_on_dataset(df)
    trained_models[file] = model

Shape for Bach_Prelude_and_Fugue_in_A-flat_major_BWV862_gCL5Zvnt0TU_a_corresp.txt: (765, 10)
DataFrame for Bach_Prelude_and_Fugue_in_A-flat_major_BWV862_gCL5Zvnt0TU_a_corresp.txt:
  alignID alignOntime alignSitch alignPitch alignOnvel refID refOntime  \
0       0    0.759375        G#3         56         66     0  0.000000   
1       1    0.763542         C4         60         62     2  0.000000   
2       2    0.763542        Eb4         63         74     1  0.000000   
3       3    1.039583        G#4         68         68     3  0.300000   
4       4    1.255208        G#4         68         85     5  0.600000   

  refSitch refPitch refOnvel  
0      G#3       56       64  
1       C4       60       64  
2      Eb4       63       64  
3      G#4       68       64  
4      G#4       68       64  
--------------------------------------------------
Shape for Bach_Prelude_and_Fugue_in_F-sharp_major_BWV_858_lJCpUW1Q1yc_a_corresp.txt: (555, 10)
DataFrame for Bach_Prelude_and_Fugue_in_F-s

KeyboardInterrupt: 